recognition model 2

In [2]:
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import matplotlib.pyplot as plt
import os

In [3]:
# Define paths
data_dir = Path(r"C:\Users\MSII\Desktop\breed recognition data set\Indian_bovine_breeds")  # Replace with actual path
img_size = (224, 224)
batch_size = 32

# Load dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

# Class names
class_names = train_ds.class_names
num_classes = len(class_names)

# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

Found 5947 files belonging to 41 classes.
Using 4758 files for training.
Found 5947 files belonging to 41 classes.
Using 1189 files for validation.


In [4]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [5]:
base_model = MobileNetV2(input_shape=img_size + (3,),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False  # Freeze base

inputs = layers.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)

In [8]:
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 730ms/step - accuracy: 0.4993 - loss: 1.6515

149/149 ━━━━━━━━━━━━━━━━━━━━ 137s 898ms/step - accuracy: 0.4941 - loss: 1.6865 - val_accuracy: 0.4062 - val_loss: 2.0530
Epoch 2/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 605ms/step - accuracy: 0.5288 - loss: 1.5614

149/149 ━━━━━━━━━━━━━━━━━━━━ 110s 737ms/step - accuracy: 0.5137 - loss: 1.6155 - val_accuracy: 0.4239 - val_loss: 2.0145
Epoch 3/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 113s 753ms/step - accuracy: 0.5223 - loss: 1.5952 - val_accuracy: 0.4348 - val_loss: 2.0218
Epoch 4/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 111s 740ms/step - accuracy: 0.5340 - loss: 1.5649 - val_accuracy: 0.4146 - val_loss: 2.0263
Epoch 5/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 587ms/step - accuracy: 0.5411 - loss: 1.5008

149/149 ━━━━━━━━━━━━━━━━━━━━ 108s 722ms/step - accuracy: 0.5223 - loss: 1.5567 - val_accuracy: 0.4390 - val_loss: 1.9898
Epoch 6/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 110s 733ms/step - accuracy: 0.5290 - loss: 1.5620 - val_accuracy: 0.4348 - val_loss: 2.0029
Epoch 7/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 0s 593ms/step - accuracy: 0.5375 - loss: 1.5171

149/149 ━━━━━━━━━━━━━━━━━━━━ 108s 725ms/step - accuracy: 0.5282 - loss: 1.5426 - val_accuracy: 0.4558 - val_loss: 1.9739
Epoch 8/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 110s 736ms/step - accuracy: 0.5410 - loss: 1.5326 - val_accuracy: 0.4315 - val_loss: 2.0040
Epoch 9/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 107s 717ms/step - accuracy: 0.5418 - loss: 1.4920 - val_accuracy: 0.4180 - val_loss: 2.0602
Epoch 10/10
149/149 ━━━━━━━━━━━━━━━━━━━━ 106s 710ms/step - accuracy: 0.5427 - loss: 1.4945 - val_accuracy: 0.4365 - val_loss: 1.9940
